In [1]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys

sys.path.append('../src')
sys.path.append('../tests')
from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.neural import GeneClassifier
from models.lib.data import *

Let's do some simple preprocessing for training, and then test our library

In [15]:
le = LabelEncoder()

labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
# labels = labels[labels['CellType'].isin(['retinal rod cell type B', 'retinal rod cell type A'])]
# labels = labels[labels['CellType'] != 'unannotated']
labels['class_label'] = le.fit_transform(labels['CellType'])

labels.index.name = 'cell'
labels = labels.iloc[0:5000, :]
labels.to_csv('../data/retina/retina_labels_numeric.csv')

# label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# # label_df

# labels.to_csv('../data/retina/retina_labels_numeric.csv')

label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv')
label_df

,cell,index,Expressed Genes,UMI Count,Percent Mitochond.,BroadCellType,CellType,Donor,class_label
0,0,0024369980fd003553cbc9dfe29f7f95,2351,6060.0,4.125413,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,9
1,1,002f621cb1c922521e488d9cbf8b707d,5868,28695.0,3.896149,unannotated,unannotated,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,12
2,2,0037f1b36684cb59b84d3585ca55ff69,950,1507.0,0.597213,retinal rod cell,retinal rod cell type B,427c0a62-9baf-42ab-a3a3-f48d10544280,10
3,3,00390952646f52d11a9ab9bba7d6ac51,961,1962.0,7.543323,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,9
4,4,005b3351658380695a5dc46c384d72d7,858,1384.0,0.144509,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,10
...,...,...,...,...,...,...,...,...,...
4995,4995,fe3024f42c4a82e5dcb5c31e3c930ecb,8633,54861.0,1.656915,unannotated,unannotated,427c0a62-9baf-42ab-a3a3-f48d10544280,12
4996,4996,fe324f69654a898c27a875a0174a1100,517,771.0,0.389105,unannotated,unannotated,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,12
4997,4997,fe5d128cf902eea5aae5149fbc63a061,1056,1763.0,0.283607,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,10
4998,4998,fe6e08381b583066f5d573b142851a0e,935,1379.0,0.290065,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,10


In [3]:
datafiles = ['../data/retina/retina_T.csv']
labelfiles = ['../data/retina/retina_labels_numeric.csv']

module = DataModule(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    batch_size=4,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    index_col='cell',
)

# total_class_weights(labelfiles, 'class_label')

In [4]:
model = GeneClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    weights=total_class_weights(['../data/retina/retina_labels_numeric.csv'], 'class_label'),
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.StepLR,
        'step_size': 20,
        'gamma': 0.001,
    }
)

Model initialized. input_dim = 37475, output_dim = 13. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


In [5]:
# model.configure_optimizers()

In [6]:
# model(next(iter(module.trainloader)))

In [8]:
trainer, genmodel, module = generate_trainer(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=4,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    weighted_metrics=False,
    normalize=True,
    weights=total_class_weights(['../data/retina/retina_labels_numeric.csv'], 'class_label'),
    wandb_name='local-retina-model',
        optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.StepLR,
        'step_size': 20,
        'gamma': 0.001,
    }
)

genmodel.scheduler_params

Device is cpu
../data/retina/retina_T.csv exists, continuing...
../data/retina/retina_labels_numeric.csv exists, continuing...



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 37475, output_dim = 13. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


{'scheduler': torch.optim.lr_scheduler.StepLR, 'step_size': 20, 'gamma': 0.001}

In [18]:
trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.5 M 
----------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.916     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of 

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [8]:
trainloader.dataset[0:50]

[(tensor([0., 0., 1.,  ..., 0., 0., 0.]), 2),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 4),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 1., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 1.,  ..., 0., 0., 0.]), 7),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 10),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 1., 0.,  ..., 0., 0., 0.]), 10),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 1., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 7),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 10),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 9),
 (tensor([0., 0., 0.,  ..., 0.,

In [9]:
def _test_first_n_samples(n, datafile, labelfile):
    data = GeneExpressionData(
        datafile, 
        labelfile, 
        'class_label', 
        skip=3,
        index_col='cell'
    )
    cols = data.columns
    
    # Generate dict with half precision values to read this into my 16gb memory
    dtype_cols = dict(zip(cols, [np.float32]*len(cols)))
    
    data_df = pd.read_csv(datafile, nrows=2*n, header=1, dtype=dtype_cols) # Might need some extras since numerical index drops some values
    label_df = pd.read_csv(labelfile, nrows=n)

    similar = []
    for i in range(n):
        datasample = data[i][0]

        dfsample = torch.from_numpy(data_df.loc[label_df.loc[i, 'cell'], :].values).float()
        isclose = all(torch.isclose(datasample, dfsample))
        similar.append(isclose)
    
    print(f"First {n=} columns of expression matrix is equal to GeneExpressionData: {all(p for p in similar)}")

    assert (all(p for p in similar))

_test_first_n_samples(100, '../data/retina/retina_T.csv', '../data/retina/retina_labels_numeric.csv')


First n=100 columns of expression matrix is equal to GeneExpressionData: True


In [10]:
idk = module.trainloader.dataset._labeldf['cell']
idk

0        507
1        840
2       4529
3       3166
4        119
        ... 
3195    5101
3196    4513
3197    1811
3198    2809
3199    3403
Name: cell, Length: 3200, dtype: int64

In [11]:
df = pd.read_csv('../data/retina/retina_labels_numeric.csv')
labels = df.loc[:, 'class_label']

In [12]:
df.loc[idk, :]

KeyError: '[5419, 5261, 5888, 5228, 5865, 5907, 5443, 5690, 5065, 5521, 5711, 5278, 5066, 5183, 5886, 5341, 5190, 5291, 5352, 5442, 5851, 5127, 5571, 5671, 5402, 5908, 5217, 5689, 5450, 5826, 5078, 5901, 5968, 5679, 5801, 5399, 5877, 5936, 5077, 5041, 5320, 5727, 5417, 5062, 5853, 5663, 5566, 5611, 5006, 5416, 5439, 5311, 5444, 5013, 5875, 5568, 5536, 5724, 5104, 5703, 5043, 5507, 5187, 5328, 5838, 5499, 5501, 5687, 5426, 5956, 5008, 5007, 5286, 5932, 5095, 5684, 5622, 5661, 5230, 5103, 5146, 5856, 5791, 5699, 5233, 5691, 5281, 5542, 5781, 5138, 5162, 5576, 5203, 5831, 5764, 5837, 5418, 5751, 5782, 5969, 5120, 5510, 5345, 5780, 5909, 5317, 5587, 5806, 5064, 5946, 5111, 5462, 5292, 5963, 5213, 5547, 5174, 5383, 5609, 5563, 5754, 5493, 5768, 5743, 5196, 5000, 5842, 5539, 5160, 5342, 5159, 5720, 5506, 5735, 5939, 5668, 5833, 5238, 5861, 5454, 5169, 5893, 5516, 5825, 5260, 5744, 5089, 5974, 5855, 5753, 5897, 5583, 5215, 5591, 5766, 5274, 5721, 5965, 5172, 5221, 5657, 5386, 5059, 5824, 5258

In [ ]:
from sklearn.model_selection import train_test_split 

trainsplit, valsplit = train_test_split(labels, stratify=labels)

trainsplit, valsplit

In [ ]:
df.loc[trainsplit, :]

In [ ]:
df.to_csv('test.csv')

pd.read_csv('test.csv')

In [ ]:
df.to_csv('')